In [1]:
import os

import pandas

import dask.dataframe as dd

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
from tqdm.dask import TqdmCallback

cb = TqdmCallback(desc="global")
cb.register()

from tqdm import tqdm_notebook, tqdm

tqdm_notebook().pandas()
tqdm.pandas(desc="global")

C:\Users\AJ Stangl\AppData\Local\Temp\ipykernel_7852\2836536399.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [3]:
from tqdm import tqdm

class InnerProgressBar(tqdm):
	def __init__(self, total, desc):
		super().__init__(desc=desc)
		self.total = total
		self.current = 0

	def update_to(self):
		self.update(self.current)

In [4]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_raw_data_path = data_path + "raw_data.parquet"

parquet_process_data_path = data_path + "processed_data.parquet"

In [5]:
os.makedirs(data_path, exist_ok=True)

print(f"Data Path:\t{data_path}\n\nExists:\t{os.path.exists(data_path)}")

Data Path:	/data/parquet/

Exists:	True


In [6]:
pandas_df = None
if os.path.exists("foo"):
	print("Loading from parquet")
	pandas_df = pandas.read_parquet(parquet_raw_data_path)


else:
	print("Loading from table")
	raw_data = table_adapter.get_all_entities(table_name)
	pandas_df = pandas.DataFrame(list(raw_data))
	print(f"Saving to parquet {parquet_process_data_path}")
	pandas_df.to_parquet(parquet_raw_data_path)

Loading from table
Saving to parquet /data/parquet/processed_data.parquet


In [7]:
print(f"Initial Dataframe Shape:\t{pandas_df.shape}")

display(pandas_df)

Initial Dataframe Shape:	(11394, 19)


,PartitionKey,RowKey,author,caption,curated,exists,exits,flair,hash,id,image,image_name,permalink,small_image,subreddit,text,updated_caption,updated_path,url
0,training,1000cej,OtterlyFoxy,a city street filled with lots of tall buildings,False,True,True,NaN,7a8d96e378c15c8ab8440ac311f12c11,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,CityPorn,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\4emw5uldib9a1.jpg,https://i.redd.it/4emw5uldib9a1.jpg
1,training,1000d16,princessxo699,a woman sitting on a table holding a pink flower,False,True,True,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,https://i.imgur.com/GgFEagO.jpg
2,training,1000fg0,BlkBrd1312,a woman in a bikini with a cell phone in her hand,False,True,True,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,HotGirlNextDoor,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,https://i.redd.it/nwa7hts2jb9a1.jpg
3,training,1000glf,toolate_sharkbait,a beautiful young woman in a black dress posin...,False,True,True,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,AmIhotAF,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,https://i.redd.it/4xyb1vgbjb9a1.jpg
4,training,1000j1n,trent8051,a collage of photos showing a man in a suit an...,False,True,NaN,NaN,1dec3dabb5e46cde01855d06089c287a,1000j1n,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,/r/greentext/comments/1000j1n/anon_wants_elon_...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,greentext,Anon wants Elon cut,a collage of photos showing a man in a suit an...,NaN,https://i.redd.it/3mewbe0wjb9a1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11389,training,zzzeoi,ComprehendApprehend,A man wearing a black shirt and black pants is...,False,True,NaN,NaN,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,D:\data\images\greentext\bgho6WK.jpg,bgho6WK.jpg,/r/greentext/comments/zzzeoi/anon_does_a_littl...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,greentext,Anon does a little trolling,a series of photos showing a man in a suit,NaN,https://i.imgur.com/bgho6WK.jpg
11390,training,zzzg13,AdConsistent3184,A little girl in a pink dress is sitting on a ...,False,False,False,NaN,ba89401054b09d3ff2b608705f8a7110,zzzg13,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,3rv0ql34bb9a1.jpg,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,,SFWRedheads,Cute but Deadly 😋,NaN,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,https://i.redd.it/3rv0ql34bb9a1.jpg
11391,training,zzzlbf,Slipstream_Valet,a man in a red shirt and a red hat,False,True,NaN,NaN,5007b937974ae333022c0c91b795ca09,zzzlbf,/data/images/trippinthroughtime/arCpzQ0.jpg,arCpzQ0.jpg,/r/trippinthroughtime/comments/zzzlbf/he_didnt...,/data/images/trippinthroughtime/thumbnail/arCp...,trippinthroughtime,He didn't shed light on the topic I guess.,a man in a red shirt and a woman in a red shirt,NaN,https://i.imgur.com/arCpzQ0.jpg
11392,training,zzzlyl,Particular-Chip2355,Error in captioning image,False,False,True,NaN,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,Minquan_Bridge2017_TAIWAN.jpg,/r/

In [8]:
print("Normalizing Dataframe For Processing")

subreddit = pandas_df["subreddit"]
image = pandas_df["image_name"]
text = pandas_df["text"]
hash_value = pandas_df["hash"]
original_caption = pandas_df["caption"]
thumbnail_caption = pandas_df["updated_caption"]
comment_id = pandas_df["id"]
all_normalized_frame = pandas.DataFrame(
	{
		"subreddit": subreddit,
		"file_name": image,
		"text": text,
		"original_caption": original_caption,
		"thumbnail_path": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}",
										  axis=1),
		"thumbnail_caption": thumbnail_caption,
		"thumbnail_exists": pandas_df.apply(
			lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"original_image": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\{x['image_name']}", axis=1),
		"original_image_exists": pandas_df.apply(
			lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"hash": hash_value,
		"id": comment_id
	}
)
print(f"Normalized Dataframe Shape:\t{all_normalized_frame.shape}")
print(f"Saving to parquet {parquet_process_data_path}")
all_normalized_frame.to_parquet(parquet_process_data_path)

Normalizing Dataframe For Processing
Normalized Dataframe Shape:	(11394, 11)
Saving to parquet /data/parquet/processed_data.parquet


In [9]:
print(f"Reading from parquet {parquet_process_data_path}")
all_normalized_frame = pandas.read_parquet(parquet_process_data_path)
display(all_normalized_frame)

Reading from parquet /data/parquet/processed_data.parquet


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
1,SFWNextDoorGirls,GgFEagO.jpg,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,a woman sitting on a table holding a pink flower,False,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,False,9951b4f82caeb8ba2bd9f79f8d422450,1000d16
2,HotGirlNextDoor,nwa7hts2jb9a1.jpg,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,a woman in a bikini with a cell phone in her hand,False,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,False,be8dd55e34216bec1e15e03fa296eacc,1000fg0
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
...,...,...,...,...,...,...,...,...,...,...,...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi
11390,SFWRedheads,3rv0ql34bb9a1.jpg,Cute but Deadly 😋,A little girl in a pink dress is sitting on a ...,D:\data\images\SFWRedheads\thumbnail\3rv0ql34b...,NaN,False,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,False,ba89401054b09d3ff2b608705f8a7110,zzzg13
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red shirt and a red hat,D:\data\images\trippinthroughtime\thumbnail\ar...,a man in a red shirt and a woman in a red shirt,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf
11392,CityPorn,Minquan_Bridge2017_TAIWAN.jpg,"Taipei, Taiwan",Error in captioning image,D:\data\images\CityPorn\thumbnail\Minquan_Brid...,NaN,False,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,False,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl


In [10]:
print("Filtering Subreddits with Images")
filtered_on_exist = all_normalized_frame[
	(all_normalized_frame["thumbnail_exists"] == True) &
	(all_normalized_frame["original_image_exists"] == True)
	]

total_images_display = filtered_on_exist.groupby("subreddit").size().reset_index(name="count")
display(total_images_display.sort_values("count", ascending=False))
print(f"Total Records {total_images_display['count'].sum()}")

Filtering Subreddits with Images


,subreddit,count
3,EarthPorn,1178
2,CityPorn,1051
4,Faces,976
7,SFWRedheads,834
9,greentext,693
13,spaceporn,604
6,SFWNextDoorGirls,602
10,memes,592
14,trippinthroughtime,275
12,sfwpetite,273


Total Records 7679


In [11]:
print("Data With Known Images")
display(filtered_on_exist)

Data With Known Images


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,a nighttime view of a large rock formation,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,a nighttime view of a large waterfall,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,a boat sitting on top of a lush green field,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,a boat sitting on top of a lush green field,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd
...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,a large amount of light is reflected in the sky,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,a large group of lights on a building,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,a blue and white photo of a blue and white obj...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,a blurry photo of a blue and white object,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5
11389,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red shirt and a red hat,D:\data\images\trippinthroughtime\thumbnail\ar...,a man in a red shirt and a woman in a red shirt,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf


In [12]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2TokenizerFast
import torch
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model = VisionEncoderDecoderModel.from_pretrained(pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained(
	pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_model_name_or_path="D:\\models\\vit-gpt2-image-captioning")


def caption_image(image_path: str, index: int) -> str:
	try:
		device = torch.device(f"cpu")

		model.to(device)

		img = Image.open(image_path)
		if img.mode != 'RGB':
			img = img.convert(mode="RGB")

		pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values

		pixel_values = pixel_values.to(device)

		max_length = 50
		num_beams = 1

		# get model prediction
		output_ids = model.generate(pixel_values, num_beams=num_beams, max_length=max_length)

		# decode the generated prediction
		predictions = tokenizer.decode(output_ids[0], skip_special_tokens=True)
		return predictions

	except Exception as e:
		print(f":: Error in caption_image: {e}")
		return "bruh"
	finally:
		pass

In [31]:
from transformers import pipeline
text_pipe = pipeline('text-generation', model='daspartho/prompt-extend', device=0)

In [33]:
def create_captions(c):
	progress.update()
	old_caption = c['original_caption']
	if old_caption != "" and old_caption is not None and old_caption != "bruh":
		return old_caption
	else:
		return text_pipe(c['text'], num_return_sequences=1, max_length=50)[0]["generated_text"]

In [14]:
def wrapper_for_blip(row: object) -> str:
	bruh = "bruh"
	try:
		caption = row["original_caption"]
		image_path = row["original_image"]
		foo = row.__dict__['_name']  # Fucking silly
		row_index = int(foo)
		progress.update()

		if caption and len(caption) > 5:
			return caption
		else:
			return caption_image(image_path, row_index)
	except Exception as e:
		print(e)
		return bruh

In [15]:
temp = filtered_on_exist.copy()
ddf = dd.from_pandas(temp, npartitions=12)

In [16]:
progress: tqdm = InnerProgressBar(len(temp), "Captioning-Primary-Images")
print(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: wrapper_for_blip(x), axis=1, meta=('str', object)).compute()

Captioning-Primary-Images: 0it [00:00, ?it/s]

Total Images: 7679


global:   0%|          | 0/12 [00:00<?, ?it/s]

Captioning-Primary-Images:  95%|█████████▌| 7324/7679 [3:35:13<45:21,  7.66s/it]   

In [17]:
print("=== Before Drop ===")
display(temp)
temp.rename(columns={"original_caption": "original_caption_old"}, inplace=True)

temp.rename(columns={"new_column": "original_caption"}, inplace=True)
temp.drop(columns=["original_caption_old"], inplace=True)
print("=== After Drop ===")
display(temp)

=== Before Drop ===


,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,a beautiful young woman in a black dress posin...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a collage of photos showing a man in a suit an...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,a nighttime view of a large rock formation,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,a nighttime view of a large waterfall,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,a boat sitting on top of a lush green field,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,a boat sitting on top of a lush green field,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field
...,...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,a large amount of light is reflected in the sky,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,a large group of lights on a building,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,a blue and white photo of a blue and white obj...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,a blurry photo of a blue and white object,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,A man wearing a black shirt and black pants is...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red shirt and a red hat,D:\data\images\trippinthroughtime\thumbnail\ar...,a man in a red shirt and a woman in a red shirt,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat


=== After Drop ===


,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,a nighttime view of a large waterfall,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,a boat sitting on top of a lush green field,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field
...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,a large group of lights on a building,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,a blurry photo of a blue and white object,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,a man in a red shirt and a woman in a red shirt,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat


In [18]:
print(f"Saving to parquet {parquet_process_data_path}")
temp.to_parquet(parquet_process_data_path)
del temp

Saving to parquet /data/parquet/processed_data.parquet


In [19]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")
processed_with_captions = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions)

Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,a city street filled with lots of tall buildings,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,a beautiful young woman in a black dress posin...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,a collage of photos showing a man in a suit an...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,a nighttime view of a large waterfall,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,a boat sitting on top of a lush green field,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field
...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,a large group of lights on a building,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,a blurry photo of a blue and white object,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,a series of photos showing a man in a suit,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,a man in a red shirt and a woman in a red shirt,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat


In [20]:
temp = processed_with_captions.copy()
ddf = dd.from_pandas(temp, npartitions=6)

In [34]:
progress = InnerProgressBar(len(temp), "Captioning-Enriched-Images")
print(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: create_captions(x), axis=1, meta=('str', object)).compute()


upload to cloud: 100%|██████████| 7679/7679 [18:13<00:00,  7.02it/s]


Total Images: 7679


global:   0%|          | 0/12 [00:00<?, ?it/s]


Captioning-Enriched-Images:   0%|          | 12/7679 [00:00<07:57, 16.06it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Captioning-Enriched-Images:   0%|          | 12/7679 [00:00<07:57, 16.06it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.

Captioning-Enriched-Images:   0%|          | 12/7679 [00:00<07:57, 16.06it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
D:\workspaces\General\venv\lib\site-packages\transformers\pipelines\base.py:1043: User

In [35]:
print("=== Before Drop ===")
display(temp)
temp.rename(columns={"thumbnail_caption": "thumbnail_caption_old"}, inplace=True)
temp.rename(columns={"new_column": "thumbnail_caption"}, inplace=True)
temp.drop(columns=["thumbnail_caption_old"], inplace=True)
print("=== After Drop ===")
display(temp)

=== Before Drop ===


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings,a city street filled with lots of tall buildin...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation,a nighttime view of a large waterfall,a nighttime view of a large rock formation me...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field,a boat sitting on top of a lush green field,a boat sitting on top of a lush green field i...
...,...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky,a large group of lights on a building,a large amount of light is reflected in the sk...
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...,a blurry photo of a blue and white object,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat,a man in a red shirt and a woman in a red shirt,a man in a red shirt and a red hat sitting wi...


=== After Drop ===


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildin...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation,a nighttime view of a large rock formation me...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field,a boat sitting on top of a lush green field i...
...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky,a large amount of light is reflected in the sk...
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat,a man in a red shirt and a red hat sitting wi...


In [36]:
print(f"Saving to parquet {parquet_process_data_path} with Updated Thumbnail Captions")
temp.to_parquet(parquet_process_data_path)
del temp

Saving to parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions


In [37]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Thumbnail Captions")
processed_with_captions_more = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions_more)

Reading from parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildin...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation,a nighttime view of a large rock formation me...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field,a boat sitting on top of a lush green field i...
...,...,...,...,...,...,...,...,...,...,...,...
11387,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky,a large amount of light is reflected in the sk...
11388,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...,a blue and white photo of a blue and white obj...
11389,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,A man wearing a black shirt and black pants is...
11391,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat,a man in a red shirt and a red hat sitting wi...


In [38]:
print("Filtering Subreddits with Images By original_caption")
filtered_captions = processed_with_captions_more[
	(processed_with_captions_more["original_caption"] != "bruh") &
	(~processed_with_captions_more["original_caption"].isna() | ~ processed_with_captions_more[
		"original_caption"].isnull())
	]

filtered_captions_display = filtered_captions.groupby("subreddit").size().reset_index(name="count")

display(filtered_captions_display.sort_values("count", ascending=False))
print(f"Total Records {filtered_captions_display['count'].sum()}")

Filtering Subreddits with Images By original_caption


,subreddit,count
3,EarthPorn,1178
2,CityPorn,1051
4,Faces,976
7,SFWRedheads,834
9,greentext,693
13,spaceporn,604
6,SFWNextDoorGirls,602
10,memes,592
14,trippinthroughtime,275
12,sfwpetite,273


Total Records 7679


In [39]:
print("Filtering Subreddits with Images By thumbnail_caption")
filtered_captions_by_thumbnail = filtered_captions[
	(processed_with_captions_more["thumbnail_caption"] != "bruh") &
	(~processed_with_captions_more["thumbnail_caption"].isna() | ~ processed_with_captions_more[
		"thumbnail_caption"].isnull())
	]

filtered_captions_by_thumbnail_display = filtered_captions_by_thumbnail.groupby("subreddit").size().reset_index(
	name="count")
display(filtered_captions_by_thumbnail_display.sort_values("count", ascending=False))
print(f"Total Records {filtered_captions_by_thumbnail_display['count'].sum()}")

Filtering Subreddits with Images By thumbnail_caption


,subreddit,count
3,EarthPorn,1178
2,CityPorn,1051
4,Faces,976
7,SFWRedheads,834
9,greentext,693
13,spaceporn,604
6,SFWNextDoorGirls,602
10,memes,592
14,trippinthroughtime,275
12,sfwpetite,273


Total Records 7679


In [40]:
print("Updating Cloud Storage With Filtered Captioned Images")
def update_cloud(row):
	try:
		_table_adapter = TableAdapter()
		partition_key = "training"
		row_key = row["id"]
		caption = row["original_caption"]
		updated_caption = row["thumbnail_caption"]
		entity = _table_adapter.get_entity("training", partition_key, row_key)
		entity["caption"] = caption
		entity["updated_caption"] = updated_caption
		_table_adapter.upsert_entity_to_table("training", entity)
		progress.update()
		return "bruh"
	except Exception as e:
		print(e)
		return "bruh"

Updating Cloud Storage With Filtered Captioned Images


In [41]:
progress = InnerProgressBar(len(processed_with_captions_more), "upload to cloud")
temp = processed_with_captions_more.copy()
ddf = dd.from_pandas(temp, npartitions=12)

Captioning-Enriched-Images: 100%|██████████| 7679/7679 [8:48:50<00:00,  4.13s/it]


In [42]:
ddf.apply(lambda x: update_cloud(x), axis=1, meta=('str', object)).compute()

global:   0%|          | 0/12 [00:00<?, ?it/s]

upload to cloud: 100%|█████████▉| 7678/7679 [02:34<00:00, 17.89it/s]

0        bruh
3        bruh
4        bruh
5        bruh
7        bruh
         ... 
11387    bruh
11388    bruh
11389    bruh
11391    bruh
11393    bruh
Name: str, Length: 7679, dtype: object

In [43]:
print("Process Complete: 8Items Updated")

Process Complete: 8Items Updated
